<a href="https://colab.research.google.com/github/rgukhui/cm4709/blob/main/Lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CM4709 Computer Vision
#Lab 04 Convolutional Neural Network

##Aims
1. Use the Keras library to build a Convolutional Neural Network (CNN).
1. Use a CNN to classify images in the Fashion MNIST dataset.


##The Fashion MNIST Dataset

In Lab 03, we used a Neural Network to classify images in the [fashion MNIST dataset](https://www.kaggle.com/datasets/zalando-research/fashionmnist).
This week, we will use a Convolutional Neural Network (CNN) to do the same job.
We will compare the performance of these 2 approaches before any tuning.

The following code is from Lab 03.
It simply loads the Fashion MNIST data and prepare the training, validation, and testing datasets.

There is no change from last week:


In [ ]:
#import libraries
import tensorflow as tf
from tensorflow import keras

#load dataset from Keras
(x_train_full,y_train_full),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()

#split full training dataset into validation and training subsets
x_valid=x_train_full[:5000]/255.0
x_train=x_train_full[5000:]/255.0
y_valid=y_train_full[:5000]
y_train=y_train_full[5000:]

print('Full training dataset (images): ',x_train_full.shape)
print('Full training dataset (labels): ',y_train_full.shape)
print('  Training dataset (images): ',x_train.shape)
print('  Training dataset (labels): ',y_train.shape)
print('  Validation dataset (images): ',x_valid.shape)
print('  Validation dataset (labels): ',y_valid.shape)
print('Testing dataset (images): ',x_test.shape)
print('Testing dataset (labels): ',y_test.shape)


##Building the Neural Network

The following code builds a neural network of 4 layers.
1. We use a [Sequential model](https://keras.io/guides/sequential_model/).
1. The input will feed into a [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/) layer.
1. This is followed by a [MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/) layer.
1. Then another [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/) layer.
1. And another [MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/) layer.
1. Then we flatten the layer.
1. Fit it into a [Dense](https://keras.io/api/layers/core_layers/dense/) layer.
1. Finally the output layer has 1 neurons, corresponding to the 10 classes in the dataset. Again, we use the [Softmax activation function](https://keras.io/api/layers/activations/#softmax-function) in the output layer.

In [ ]:
#create a neural network using Keras
model=keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3),activation='relu', padding='same',input_shape=(28,28,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3),activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3),activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Flatten())

# Densely connected layers
model.add(tf.keras.layers.Dense(128, activation='relu'))

model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(64, activation='relu'))

model.add(tf.keras.layers.Dropout(0.2))

# output layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.summary()

##Questions
Look at different layers in the CNN:
1. What is the size of the kernel in the 1st convolutional layer?
1. How many kernels do we have in the 1st conv layer?
1. How many parameters are in the 1st conv layer? What do they correspond to?
1. What is the size of the 1st pooling layer?
1. Compare the number of parameters in a NN. What do you notice?


##Preparing the Labels

There are 10 classes in the dataset.
The output layer, with 10 neurons, expect a hot-one encoding.
For example, an output label of `4` is expected in the form of a vector/array: `[0,0,0,0,1,0,0,0,0,0]`.

The following code show you the shape of the training dataset labels, and the first element value:

In [ ]:
print('y_train shape: ',y_train.shape)
print('y_train[0]: ',y_train[0])

Instead of a value of 4, we want the output in the 1-hot-encoding format. e.g. a vector [0,0,0,0,1,0,0,0,0,0]. The following code converts the output datasets into the expected format/encoding. We use a utility function [`keras.utils.categorical(...)`](https://keras.io/api/utils/python_utils/#tocategorical-function):. Note that we only do this to the output label. i.e. Y:

In [ ]:
y_train_cat=keras.utils.to_categorical(y_train)
y_test_cat=keras.utils.to_categorical(y_test)
y_valid_cat=keras.utils.to_categorical(y_valid)

print('y_train_cat.shape: ',y_train_cat.shape)
print('y_train_cat[0]: ',y_train_cat[0])

##Compiling and Training the Model

Now we can compile and train the model.
This time we are using the [Adam optimizer](https://keras.io/api/optimizers/adam/).
Loss function is [categorical_cross_entropy[(https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class).

##Questions:
1. Look at the output dataset (i.e. `y_train_cat`). Why do you think we use `categorical_crossentropy` as the loss function?

In [ ]:
# compile with adam optimizer & categorical_crossentropy loss function
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics='accuracy')

#train model
history=model.fit(x_train,y_train_cat,epochs=20,
                  validation_data=(x_valid,y_valid_cat))


##Visualising the Training

The following code (from Lab 03) shows the graphs of loss, accuracy, validation-loss, and validation-accuracy as the training progressed:

In [ ]:
import pandas as pd
import matplotlib.pyplot as pyplot

#show keys in the history
print(history.history.keys())

#plot graphs
pd.DataFrame(history.history).plot(figsize=(15,8))
pyplot.grid(True)
pyplot.gca().set_ylim(0,1)
pyplot.show()

##Question
1. What do you observe in the graphs?
1. What does this compare with the NN solution in Lab 03 on the same dataset?

##Testing the Model

Finally, we use the testing dataset to evaluate the CNN's performance on unseen data:

In [ ]:
#evaluate the model using the testing dataset
(loss,accuracy)=model.evaluate(x_test,y_test_cat)

print('Loss: ',loss)
print('Accuracy: ',round(accuracy,2)*100,'%')

##Questions

1. How does the performance of the CNN compare with the NN in Lab 03?
1. Suggest ways to improve the model's performance.

##Doing Prediction

The following code below from Lab 03 takes a random image from the testing dataset and does a prediction of its class:

In [ ]:
import numpy as np
import random

#class names in an array
class_names=['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

#get a random image from the testing dataset
(instance_count,width,height)=x_test.shape    #get dataset size
index=random.randint(0,instance_count-1)      #get an index witin dataset
image=x_test[index]                           #get image
pyplot.imshow(image,cmap='gray')              #show image

#need to add 1 dimension to array to fit input shape
oneImage=np.expand_dims(image,axis=0)
print('Image shape: ',oneImage.shape)

#predict image class
[prediction]=model.predict(oneImage)
print('Prediction output: ',prediction)

classIndex=np.argmax(prediction,axis=-1)

predictedClass=class_names[classIndex]
print('Predicted class: ',predictedClass)
print('Known class: ',class_names[y_test[index]])